In [23]:
from scipy.stats import kstest
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns

In [14]:
data = '../data/'
incomes = pd.read_csv(data + "worldincome.csv", parse_dates=True) 
incomes.describe()
incomes


,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,"728,89795",7297
1,ALB,2008,2,100,"916,66235",7297
2,ALB,2008,3,100,"1010,916",7297
3,ALB,2008,4,100,"1086,9078",7297
4,ALB,2008,5,100,"1132,6997",7297
...,...,...,...,...,...,...
11594,COD,2008,96,100,"810,6233","303,19305"
11595,COD,2008,97,100,"911,7834","303,19305"
11596,COD,2008,98,100,"1057,8074","303,19305"
11597,COD,2008,99,100,"1286,6029","303,19305"


nous allons clear le dataset voir si il y a des valeur ou 

In [41]:
incomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11599 entries, 0 to 11598
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country       11599 non-null  object
 1   year_survey   11599 non-null  int64 
 2   quantile      11599 non-null  int64 
 3   nb_quantiles  11599 non-null  int64 
 4   income        11599 non-null  object
 5   gdpppp        11399 non-null  object
dtypes: int64(3), object(3)
memory usage: 543.8+ KB


In [48]:
incomes.replace(',','.', regex=True, inplace=True)
incomes['income'] = incomes['income'].astype(float)
incomes['gdpppp'] = incomes['gdpppp'].astype(float)



country         11599
year_survey     11599
quantile        11599
nb_quantiles    11599
income          11599
gdpppp          11399
dtype: int64

In [46]:
incomes[incomes['country']=='ALB'].sort_values(by='income')
## nb quantiles toujours = 100

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,728.89795,7297.0
1,ALB,2008,2,100,916.66235,7297.0
2,ALB,2008,3,100,1010.91600,7297.0
3,ALB,2008,4,100,1086.90780,7297.0
4,ALB,2008,5,100,1132.69970,7297.0
...,...,...,...,...,...,...
95,ALB,2008,96,100,6231.09960,7297.0
96,ALB,2008,97,100,6822.31840,7297.0
97,ALB,2008,98,100,7775.89100,7297.0
98,ALB,2008,99,100,9351.75100,7297.0


In [13]:
numberOfCountry = incomes.groupby('country')
numberOfCountry.count()

,year_survey,quantile,nb_quantiles,income,gdpppp
country,,,,,
ALB,100,100,100,100,100
ARG,100,100,100,100,100
ARM,100,100,100,100,100
AUT,100,100,100,100,100
AZE,100,100,100,100,100
...,...,...,...,...,...
VEN,100,100,100,100,100
VNM,100,100,100,100,100
XKX,100,100,100,100,0


In [49]:
incomes.count()

country         11599
year_survey     11599
quantile        11599
nb_quantiles    11599
income          11599
gdpppp          11399
dtype: int64

il manque une rows car nous avons 116 pays 

In [52]:
countQuantile = incomes.pivot_table(index='country').sort_values(by='quantile').reset_index()[['country', 'quantile']]
diffQuantile = countQuantile[countQuantile['quantile'] != 50.50000]
diffQuantile

,country,quantile
115,LTU,50.59596


LTU as une valeur manquante on dois lui ajouter une valeur au dataset 

In [62]:
ltu = incomes[incomes['country']=='LTU']

##https://www.appsloveworld.com/numpy/100/21/check-the-missing-integers-from-a-range-in-python
missedQuantile = np.setdiff1d(list(range(1, 101)), list(ltu['quantile']))

missedQuantile[0]

41

il manque le quantile 41 on va faire la moyenne et l'insert dans le dataset.

In [63]:
getRowsLtu = ltu[ltu['quantile'].isin([missedQuantile[0]-1,missedQuantile[0]+1])]
getRowsLtu

,country,year_survey,quantile,nb_quantiles,income,gdpppp
6239,LTU,2008,40,100,4868.4507,17571.0
6240,LTU,2008,42,100,4895.8306,17571.0


In [64]:
meanRowLtu = getRowsLtu.pivot_table(index='country').reset_index()
meanRowLtu



,country,gdpppp,income,nb_quantiles,quantile,year_survey
0,LTU,17571.0,4882.14065,100,41,2008


In [50]:
representCountry = len(numberOfCountry)
totalCountriesInWorld = 195
populationCoveradeByDf = (representCountry / totalCountriesInWorld) * 100
populationCoveradeByDf

59.48717948717949

116 pays sont dans le DF  ce qui represente 59% de la population mondial

In [20]:
dfDate = incomes.copy()

dfDate.drop_duplicates(subset='year_survey', keep='first',inplace=True)

dfDate.sort_values(by='year_survey')

,country,year_survey,quantile,nb_quantiles,income,gdpppp
9899,SYR,2004,1,100,"143,18066",4512
3600,GHA,2006,1,100,"63,157047","1380,1182"
900,BIH,2007,1,100,"808,3991",7472
0,ALB,2008,1,100,"728,89795",7297
600,BFA,2009,1,100,"138,40517","1085,5139"
700,BGD,2010,1,100,"276,93503",1233
3900,GTM,2011,1,100,"38,463615",4367


nous avons des data depuis 2004 à 2011 cependant il nous manque l'année 2005

In [24]:
quant = incomes.copy()
quant.drop_duplicates(subset='nb_quantiles', keep='first',inplace=True)
quant

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,"728,89795",7297


De quel type de quantiles s'agit-il (quartiles, déciles, etc.) ? Centile


In [25]:
incomes.describe()

,year_survey,quantile,nb_quantiles
count,11599.000000,11599.000000,11599.0
mean,2007.982757,50.500819,100.0
std,0.909633,28.868424,0.0
min,2004.000000,1.000000,100.0
25%,2008.000000,25.500000,100.0
50%,2008.000000,51.000000,100.0
75%,2008.000000,75.500000,100.0
max,2011.000000,100.000000,100.0


In [33]:
incomes.loc[incomes['nb_quantiles'] != 100]

,country,year_survey,quantile,nb_quantiles,income,gdpppp


In [40]:
incomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11599 entries, 0 to 11598
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country       11599 non-null  object
 1   year_survey   11599 non-null  int64 
 2   quantile      11599 non-null  int64 
 3   nb_quantiles  11599 non-null  int64 
 4   income        11599 non-null  object
 5   gdpppp        11399 non-null  object
dtypes: int64(3), object(3)
memory usage: 543.8+ KB
